In [1]:
import tensorflow as tf

In [3]:
import numpy as np
import math
from sklearn.metrics import confusion_matrix
import time

In [4]:
#convolutional layer 1
#when working with high dimensional inputs like images it is impractical to use a traditional feed forwrd nn
#instead we connect each neuron to a local region of the input
#make filters

filter_size1= 5    #convolution filters are 5 x 5
num_filters1= 16   #there are 16 5x5 filters

filter_size2= 5    #convolutions 5x5
num_filters= 36    # there are 36 5x5 filters

#fully connected layer

fc_size = 128 #number of neurons in fully connected layer

In [6]:
from tensorflow.examples.tutorials.mnist import input_data


In [7]:
data = input_data.read_data_sets('data/MNIST', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/MNIST\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [8]:
#we know that MNIST images are 28x28
img_size = 28

img_size_flat = img_size*img_size

#tuple with height and width of images used to reshape arrays

img_shape = (img_size,img_size)

#number of color channels...since it's grayscale this value will be 1
num_channels = 1

#number of classed is 10...one for each digit

num_classes = 10

In [18]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [20]:
images = data.test.images[0:9]
cls_true = data.test.cls[0:9]
plot_images(images=images, cls_true=clas_true)

AttributeError: 'DataSet' object has no attribute 'cls'

In [ ]:
def new_biases(length):
    #same as y intercept
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [ ]:
#has pooling and ReLU built into it...think of it as one block
def new_conv_layer(input, num_input_channels,filter_size,num_filters,use_pooling=True)

#shape of the tensorflow filter weights

shape = [filter_Size,filter_size,num_input_channels,num_filters]

#create new weights with the given shape
weights = new_weights(shape=shape)

#create new biases, one for each filter

biases= new_biases(length=num_filters)

#create the TF operation for convolution, strides set to 1 in all dimensions
#first and last must be 1, first is for image number and last is inpu_channel
#padding adds 0's to the sides so the outputs will be the same

layer = tf.nn.conv2d(input=input
                    filter=weights
                    strides = [1,1,1,1]
                    padding = 'SAME')
#add the biases to the results of the convolution
#now a bias value is added to each filter-channel
layer+=biases

#use pooling to downsample

if use_pooling:
    #2x2 maxpooling...means we take the largest value in the 2x2 window and
    #and select it...then we move 2 pixels to the next window
    layer= tf.nn.max_pool(value= layer
                         ksize=[1,2,2,1]
                         strides=[1,2,2,1]
                         padding='SAME')
    #adds non-linearity to the model by changing negative values to 0
    layer = tf.nn.relu(layer)
    
    #ReLU is normally executed before pooling
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    return layer, weights

In [ ]:
def flatten_layer(layer):
    #the shape of the uinput layer is assumed to be:
    #layer_shape == [num_images, img_height, img_width, num_channels]
    #,we can use a function from TF to calulate the features
    num_features= layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    
    layer_flat = tf.reshape(layer, [-1, num_features])
    
    return layer_flat, num_features
    

In [ ]:
def new_fc_layer(input,
                 num_imputs #num imputs from the previous layer
                 num_outputs 
                 use_relu=True)
    #create new weights and biases
    weights= new_weights(shape=[num_imputs,num_outputs])
    biases = new_biases(length=num_outputs)
    
    #calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values
    layer= tf.matmul(input,weights) + biases
    
    #use ReLU
    
    if use_relu:
        layer = tf.nn.relu(layer)

In [ ]:
#placeholder variables

x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [ ]:
#Convolutional Layer 1
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image
                   num_input_channels=num_channels
                   filter_size=filter_size1
                   num_filters=num_filters1
                   use_pooling=True)

In [ ]:
#Convolutional Layer2
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)

In [ ]:
#flatten
layer_flat, num_features = flatten_layer(layer_conv2)

In [ ]:
#Fully Connected Layer 1

#Add a fully-connected layer to the network. The input is the flattened layer from the previous convolution. 
#The number of neurons or nodes in the fully-connected layer is fc_size. 
#ReLU is used so we can learn non-linear relations.
layer_fc1 = new_fc_layer(input=layer_flat,
                        num_inputs=num_features,
                        num_outputs=fc_size,
                        use_relu=True)

In [ ]:
#Fully Connected Layer2
#Add another fully-connected layer that outputs vectors of length 10 for determining which of the 10 classes the input image belongs to. 
#Note that ReLU is not used in this layer.
layer_fc2 = new_fc_layer(input=layer_fc1,
                        num_inputs=fc_size,
                        num_outputs=num_classes,
                        use_relu=False)

In [ ]:
#we want to normalize the output of our 2nd fully connected layer so we add a softmax function
y_pred= tf.nn.softmax(layer_fc2)

#the class number is the index of the largest element
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [ ]:
#Cost Function
cross_entropy= tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                      labels=y_true)

cost = tf.reduce_mean(cross_entropy)

In [ ]:
#optimization

#This is a vector of booleans whether the predicted class equals the true class of each image.

correct_prediction = tf.equal(y_pred_cls, y_true_cls)

#This calculates the classification accuracy by first type-casting the vector of booleans to floats, 
#so that False becomes 0 and True becomes 1, and then calculating the average of these numbers.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#Run Tensorflow

session = tf.Session()

In [ ]:
session.run(tf.global_variables_initializer())

In [ ]:
#Helper function to perform opitimization iterations

train_batch_size=64

#counter for total iterations performed
total iterations = 0

In [ ]:
#Make an optimizer function

def optimize(num_iterations):
    #make sure we update the global variable not a local copy
    global total_iterations
    
    #Start-time used for printing time-usage
    start_time = time.time()
    
    for i in range(total_iterations, total_iterations +num_iterations):
        #get a batch of training samples
        #x_batch holds a batch of images and Y_true_batch are the true labels for those images
        x_batch, y_true_batch = data.train.next_batch(train_batch_size)
        
        #put the batch into the dict with names for the placeholder variables
        #in the TensorFlow Graph
        feed_dict_train = {x: x_batch, y_true: y_true_batch}
        
        #run the optimizer using this batch of training data TensorFlow assigns the variables
        #in feed_dict_train to the placeholder variables and then runs the optimizer
        
        session.run(optimizer, feed_dict=feed_dict_train)
        
        # print a status for every 100 iterations
        if i%100 == 0:
            #calculate the accuracy on the training-set
            
            acc = session.run(accuracy, feed_dict=feed_dict_train)
            
            #message for printing
            msg = 'Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}'
            
            #now print
            print(msg.format(i +1), acc)
            
            #update the total number of iterations performed
            total_iterations += num_iterations
            
            #ending time
            end_time = time.time()
            
            #difference between the start and end times
            time_dif = end_time - start_time
            
            print('Time Usage: ' + str(timedelta(seconds=int(round(time_dif)))))
    